# M6 — Proyecto final: pipeline completo + métricas + mini‑reporte (VS Code + Jupyter + Olive + ONNX Runtime)

## Objetivo
Construir un **pipeline reproducible** que:
1. Tome un modelo **ONNX baseline** (nuestro `add_const.onnx` del curso).
2. Ejecute un workflow de **Olive** (cuantización dinámica INT8 en CPU).
3. Haga una **optimización offline** adicional con ONNX Runtime (serializando el grafo optimizado).
4. Mida **tamaño** y **latencia** (benchmark simple) y valide **salida**.
5. Genere un **reporte** en `outputs/m6_report.md`.

## Prerrequisitos
- Ejecutar este notebook con el **kernel correcto** (tu `.venv` del proyecto) en VS Code.
- Tener instalado:
  - `onnxruntime`
  - `onnx`
  - `numpy`
  - `olive-ai`

> Si estás en Windows y tienes varios Pythons instalados, comprueba `sys.executable` en la celda 1.


In [ ]:
# Celda 1 — Comprobación de entorno (kernel, venv y versiones)
import sys, subprocess
from pathlib import Path

print("Python executable:", sys.executable)
print("Python version:", sys.version)

# Heurística de venv: sys.prefix != sys.base_prefix
print("sys.prefix:", sys.prefix)
print("sys.base_prefix:", sys.base_prefix)
print("Running in venv?:", sys.prefix != sys.base_prefix)

print("\n--- pip show olive-ai ---")
subprocess.run([sys.executable, "-m", "pip", "show", "olive-ai"], check=False)

print("\n--- pip show onnxruntime ---")
subprocess.run([sys.executable, "-m", "pip", "show", "onnxruntime"], check=False)


## Estructura del proyecto (recomendada)
Este notebook asume la estructura:

- `models/`  → modelos ONNX baseline
- `outputs/` → salidas de workflows y reportes

Si tu notebook está en `notebooks/`, detectamos la raíz del repo automáticamente (celda 2).


In [ ]:
# Celda 2 — Resolver rutas del proyecto (root/models/outputs)
from pathlib import Path

root = Path.cwd()
if not (root / "models").exists() and (root.parent / "models").exists():
    root = root.parent

models_dir = root / "models"
outputs_dir = root / "outputs"
models_dir.mkdir(exist_ok=True)
outputs_dir.mkdir(exist_ok=True)

print("Project root:", root)
print("models_dir:", models_dir)
print("outputs_dir:", outputs_dir)


## 0) Baseline: asegurar que existe `models/add_const.onnx`

Si vienes de M1, ya lo tienes.
Si no, esta celda lo genera (Add(X, C) -> Y con C=1.0).


In [ ]:
# Celda 3 — (Re)crear baseline ONNX si falta
import numpy as np
import onnx
from onnx import TensorProto, helper, numpy_helper

baseline_path = models_dir / "add_const.onnx"

if baseline_path.exists():
    print("OK baseline exists:", baseline_path)
else:
    # 1) IO
    X = helper.make_tensor_value_info("X", TensorProto.FLOAT, [None])
    Y = helper.make_tensor_value_info("Y", TensorProto.FLOAT, [None])

    # 2) initializer C
    C_value = np.array([1.0], dtype=np.float32)
    C = numpy_helper.from_array(C_value, name="C")

    # 3) Add
    node = helper.make_node("Add", inputs=["X", "C"], outputs=["Y"])

    # 4) graph + model
    graph = helper.make_graph([node], "add-const", [X], [Y], initializer=[C])
    opset = [helper.make_operatorsetid("", 11)]
    model = helper.make_model(graph, producer_name="m6-final", opset_imports=opset)
    model.ir_version = 11  # compatibilidad con runtimes antiguos si aplica

    onnx.checker.check_model(model)
    onnx.save_model(model, str(baseline_path))
    print("Wrote baseline:", baseline_path)

baseline_path


## 1) Métricas baseline (tamaño, latencia simple, verificación numérica)

- **Tamaño**: bytes del archivo ONNX.
- **Latencia**: medimos `session.run` en un loop (warmup + runs).
- **Verificación**: comparamos la salida con una referencia conocida para un input.


In [ ]:
# Celda 4 — Helpers de medición ORT (CPU)
import time
import numpy as np
import onnxruntime as ort

def file_size_bytes(p):
    return int(Path(p).stat().st_size)

def ort_infer_once(model_path, x):
    sess = ort.InferenceSession(str(model_path), providers=["CPUExecutionProvider"])
    input_name = sess.get_inputs()[0].name
    output_name = sess.get_outputs()[0].name
    y = sess.run([output_name], {input_name: x})[0]
    return y

def ort_latency_ms(model_path, x, warmup=20, runs=200):
    sess = ort.InferenceSession(str(model_path), providers=["CPUExecutionProvider"])
    input_name = sess.get_inputs()[0].name
    output_name = sess.get_outputs()[0].name

    for _ in range(warmup):
        sess.run([output_name], {input_name: x})

    t0 = time.perf_counter()
    for _ in range(runs):
        sess.run([output_name], {input_name: x})
    t1 = time.perf_counter()

    return (t1 - t0) * 1000.0 / runs

x = np.array([10, 20, 30], dtype=np.float32)

baseline_size = file_size_bytes(baseline_path)
baseline_y = ort_infer_once(baseline_path, x)
baseline_lat_ms = ort_latency_ms(baseline_path, x)

print("ORT version:", ort.get_version_string())
print("Baseline model:", baseline_path)
print("Baseline size (bytes):", baseline_size)
print("Baseline latency avg (ms):", baseline_lat_ms)
print("Baseline y:", baseline_y)


## 2) Workflow Olive (cuantización dinámica INT8 en CPU)

Ejecutamos `python -m olive run --config ...` desde el **mismo Python del kernel**.

Salida:
- `outputs/m6_workflow/` (output_dir)
- `outputs/m6_cache/` (cache_dir)


In [ ]:
# Celda 5 — Escribir run config (Olive) para cuantización dinámica (INT8) en CPU
import json
from pathlib import Path

run_config_path = outputs_dir / "m6_run_config.json"
output_dir = outputs_dir / "m6_workflow"
cache_dir = outputs_dir / "m6_cache"

output_dir.mkdir(exist_ok=True)
cache_dir.mkdir(exist_ok=True)

config = {
    "workflow_id": "m6_final_project_int8_cpu",
    "input_model": {
        "type": "ONNXModel",
        "config": {"model_path": str(baseline_path)},
    },
    "systems": {
        "local_system": {
            "type": "LocalSystem",
            "config": {
                "accelerators": [
                    {"device": "cpu", "execution_providers": ["CPUExecutionProvider"]}
                ]
            },
        }
    },
    "passes": {
        "opset_to_11": {"type": "OnnxOpVersionConversion", "config": {"target_opset": 11}},
        "dynamic_int8": {
            "type": "OnnxDynamicQuantization",
            "config": {"quant_mode": "dynamic", "weight_type": "QInt8"},
        },
    },
    "engine": {
        "host": "local_system",
        "target": "local_system",
        "cache_dir": str(cache_dir),
        "output_dir": str(output_dir),
        "log_severity_level": 0,
        "evaluate_input_model": False,
    },
}

run_config_path.write_text(json.dumps(config, indent=2), encoding="utf-8")
print("Wrote:", run_config_path)
print("Output dir:", output_dir)


In [ ]:
# Celda 6 — Ejecutar Olive (usa el Python del kernel)
import sys, subprocess, shlex

cmd = [sys.executable, "-m", "olive", "run", "--config", str(run_config_path)]
print("Running:", " ".join(shlex.quote(c) for c in cmd))

p = subprocess.run(cmd, text=True, capture_output=True)
print("returncode:", p.returncode)
print("\n--- stdout (first 2000 chars) ---\n", p.stdout[:2000])
print("\n--- stderr (first 2000 chars) ---\n", p.stderr[:2000])

if p.returncode != 0:
    raise RuntimeError("Olive run falló. Revisa stdout/stderr arriba.")


## 3) Localizar el modelo cuantizado generado por Olive

Buscamos recursivamente el `.onnx` más reciente dentro de `outputs/m6_workflow/`.


In [ ]:
# Celda 7 — Buscar el ONNX output más reciente
onnx_files = sorted(output_dir.rglob("*.onnx"), key=lambda p: p.stat().st_mtime, reverse=True)
print("Found .onnx:", len(onnx_files))

if not onnx_files:
    raise FileNotFoundError(f"No se encontraron .onnx en {output_dir}. Revisa el log de Olive (celda 6).")

quant_path = onnx_files[0]
print("Selected quantized model:", quant_path)
quant_path


## 4) Métricas del modelo cuantizado (tamaño, latencia, verificación)

In [ ]:
# Celda 8 — Métricas del modelo cuantizado
quant_size = file_size_bytes(quant_path)
quant_y = ort_infer_once(quant_path, x)
quant_lat_ms = ort_latency_ms(quant_path, x)

abs_diff = float(np.max(np.abs(quant_y - baseline_y)))

print("Quant model:", quant_path)
print("Quant size (bytes):", quant_size)
print("Quant latency avg (ms):", quant_lat_ms)
print("Quant y:", quant_y)
print("Max abs diff vs baseline:", abs_diff)

print("\nSize reduction (%):", (baseline_size - quant_size) * 100.0 / baseline_size)
print("Latency improvement (%):", (baseline_lat_ms - quant_lat_ms) * 100.0 / baseline_lat_ms)


## 5) Optimización OFFLINE adicional con ONNX Runtime (serializar modelo optimizado)

In [ ]:
# Celda 9 — ORT offline optimization: serializar un ONNX optimizado a disco
import onnxruntime as rt

final_dir = outputs_dir / "m6_final"
final_dir.mkdir(exist_ok=True)

final_optimized_path = final_dir / "add_const_int8_ort_optimized.onnx"

sess_options = rt.SessionOptions()
sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_EXTENDED
sess_options.optimized_model_filepath = str(final_optimized_path)

_ = rt.InferenceSession(str(quant_path), sess_options, providers=["CPUExecutionProvider"])

print("Wrote optimized model:", final_optimized_path)
final_optimized_path


## 6) Métricas del modelo FINAL (cuantizado + ORT offline optimized)

In [ ]:
# Celda 10 — Métricas del modelo final
final_size = file_size_bytes(final_optimized_path)
final_y = ort_infer_once(final_optimized_path, x)
final_lat_ms = ort_latency_ms(final_optimized_path, x)
final_abs_diff = float(np.max(np.abs(final_y - baseline_y)))

print("Final model:", final_optimized_path)
print("Final size (bytes):", final_size)
print("Final latency avg (ms):", final_lat_ms)
print("Final y:", final_y)
print("Max abs diff vs baseline:", final_abs_diff)

print("\nSize reduction vs baseline (%):", (baseline_size - final_size) * 100.0 / baseline_size)
print("Latency improvement vs baseline (%):", (baseline_lat_ms - final_lat_ms) * 100.0 / baseline_lat_ms)


## 7) Generar reporte (Markdown)

Creamos `outputs/m6_report.md` con tabla comparativa.


In [ ]:
# Celda 11 — Generar un reporte Markdown
from datetime import datetime

report_path = outputs_dir / "m6_report.md"

def fmt_bytes(n):
    n = float(n)
    for unit in ["B", "KB", "MB", "GB"]:
        if n < 1024.0:
            return f"{n:.1f} {unit}"
        n /= 1024.0
    return f"{n:.1f} TB"

lines = []
lines.append(f"# M6 Reporte — {datetime.now().isoformat(timespec='seconds')}")
lines.append("")
lines.append("## Artefactos")
lines.append(f"- Baseline: `{baseline_path}`")
lines.append(f"- Cuantizado (Olive): `{quant_path}`")
lines.append(f"- Final (ORT offline optimized): `{final_optimized_path}`")
lines.append("")
lines.append("## Métricas")
lines.append("")
lines.append("| Variante | Tamaño | Latencia avg (ms) | Max abs diff vs baseline |")
lines.append("|---|---:|---:|---:|")
lines.append(f"| Baseline | {fmt_bytes(baseline_size)} | {baseline_lat_ms:.6f} | 0.0 |")
lines.append(f"| Cuantizado (Olive) | {fmt_bytes(quant_size)} | {quant_lat_ms:.6f} | {abs_diff:.6f} |")
lines.append(f"| Final (ORT opt) | {fmt_bytes(final_size)} | {final_lat_ms:.6f} | {final_abs_diff:.6f} |")
lines.append("")
lines.append("## Comparativas (vs baseline)")
lines.append(f"- Reducción de tamaño (Cuantizado): {(baseline_size-quant_size)*100.0/baseline_size:.2f}%")
lines.append(f"- Reducción de tamaño (Final): {(baseline_size-final_size)*100.0/baseline_size:.2f}%")
lines.append(f"- Mejora de latencia (Cuantizado): {(baseline_lat_ms-quant_lat_ms)*100.0/baseline_lat_ms:.2f}%")
lines.append(f"- Mejora de latencia (Final): {(baseline_lat_ms-final_lat_ms)*100.0/baseline_lat_ms:.2f}%")
lines.append("")
lines.append("## Nota")
lines.append("- La latencia aquí es un micro‑benchmark simple en Python.")

report_path.write_text("\n".join(lines), encoding="utf-8")
print("Wrote report:", report_path)
report_path


## (Opcional) Packaging de artefactos con Olive (Zipfile)

Olive puede empaquetar artefactos en un ZIP usando `packaging_config` en la sección `engine`.


In [ ]:
# Celda 12 — (Opcional) preparar un run config con packaging_config (NO ejecuta)
packaged_config_path = outputs_dir / "m6_run_config_packaged.json"
packaged = dict(config)
packaged["engine"] = dict(config["engine"])
packaged["engine"]["packaging_config"] = {"type": "Zipfile", "name": "M6_OutputModels"}

packaged_config_path.write_text(json.dumps(packaged, indent=2), encoding="utf-8")
print("Wrote:", packaged_config_path)
print("Para ejecutar:")
print(f"  {sys.executable} -m olive run --config {packaged_config_path}")


## Verificación (qué debe salir bien)
- `outputs/m6_workflow/` contiene salidas del workflow de Olive.
- `outputs/m6_final/add_const_int8_ort_optimized.onnx` existe.
- `outputs/m6_report.md` existe y resume las métricas.

## Errores comunes y diagnóstico
1) **Olive se ejecuta con otro Python**
   - Síntoma: logs apuntan a otro `...Python...` distinto al `.venv`.
   - Solución: usa `sys.executable -m olive ...` y/o selecciona el kernel correcto.

2) **No se generan `.onnx` en output_dir**
   - Revisa `stdout/stderr` de la celda 6.
   - Comprueba que `baseline_path` existe y es válido.

3) **ORT falla al cargar el ONNX**
   - Puede ser incompatibilidad de IR/opset. Aquí fijamos `opset=11` e `ir_version=11` en el baseline.

## Fin del curso
Siguiente salto: sustituir el modelo de juguete por uno real (transformer), aplicar passes específicos y definir métricas de calidad/latencia representativas.
